## 0. 運行 SQL & 資料前處理

In [ ]:
!pip install duckdb

In [ ]:
import kagglehub

# 下載 Kaggle 資料集
path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/olistbr/brazilian-ecommerce/versions/2


In [ ]:
!ls /root/.cache/kagglehub/datasets/olistbr/brazilian-ecommerce/versions/2

olist_customers_dataset.csv	  olist_orders_dataset.csv
olist_geolocation_dataset.csv	  olist_products_dataset.csv
olist_order_items_dataset.csv	  olist_sellers_dataset.csv
olist_order_payments_dataset.csv  product_category_name_translation.csv
olist_order_reviews_dataset.csv


In [ ]:
import duckdb
import pandas as pd

# 方便後續引用，定義資料夾路徑
folder_path = '/root/.cache/kagglehub/datasets/olistbr/brazilian-ecommerce/versions/2/'

customers = folder_path + 'olist_customers_dataset.csv'
order_items = folder_path + 'olist_order_items_dataset.csv'
orders = folder_path + 'olist_orders_dataset.csv'
products = folder_path + 'olist_products_dataset.csv'
geolocation = folder_path + 'olist_geolocation_dataset.csv'

In [ ]:
# 查詢前 5 筆資料
df_customers_head = duckdb.query(f"""
    SELECT *
    FROM '{customers}'
    LIMIT 5
""").to_df()
df_customers_head

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [ ]:
#大致查看整體資料集
df_customers_orders_items = duckdb.query(f"""
    SELECT
        customer.customer_id,
        customer.customer_unique_id,
        customer.customer_zip_code_prefix,
        orders.order_id,
        orders.order_purchase_timestamp,
        order_items.product_id,
        order_items.price,
        order_items.freight_value
    FROM
        '{customers}' AS customer
    JOIN
        '{orders}' AS orders
    ON
        customer.customer_id = orders.customer_id
    JOIN
        '{order_items}' AS order_items
    ON
        order_items.order_id = order_items.order_id
    LIMIT 10
""").to_df()

df_customers_orders_items


,customer_id,customer_unique_id,customer_zip_code_prefix,order_id,order_purchase_timestamp,product_id,price,freight_value
0,9ef432eb6251297304e76186b10a928d,7c396fd4830fd04220f754e42b4e5bff,03149,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,110adb18eeba9a0984478ee3c9302e16,179.49,9.45
1,b0830fb4747a6c6d20dea0b8c802d7ef,af07308b275d755c9edb36a90c618231,47813,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,110adb18eeba9a0984478ee3c9302e16,179.49,9.45
2,41ce2a54c0b03bf3443c3d931a367089,3a653a41f6f9fc3d2a113cf8398680e8,75265,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,110adb18eeba9a0984478ee3c9302e16,179.49,9.45
3,f88197465ea7920adcdbec7375364d82,7c142cf63193a1473d2e66489a9ae977,59296,949d5b44dbf5de918fe9c16f97b45f8a,2017-11-18 19:28:06,110adb18eeba9a0984478ee3c9302e16,179.49,9.45
4,8ab97904e6daea8866dbdbc4fb7aad2c,72632f0f9dd73dfee390c9b22eb56dd6,09195,ad21c59c0840e6cb83a9ceb5573f8159,2018-02-13 21:18:39,110adb18eeba9a0984478ee3c9302e16,179.49,9.45
5,503740e9ca751ccdda7ba28e9ab8f608,80bb27c7c16e8f973207a5086ab329e2,86320,a4591c265e18cb1dcee52889e2d8acc3,2017-07-09 21:57:05,110adb18eeba9a0984478ee3c9302e16,179.49,9.45
6,ed0271e0b7da060a393796590e7b737a,36edbb3fb164b1f16485364b6fb04c73,98900,136cce7faa42fdb2cefd53fdc79a6098,2017-04-11 12:22:08,110adb18eeba9a0984478ee3c9302e16,179.49,9.45
7,9bdf08b4b3b52b5526ff42d37d47f222,932afa1e708222e5821dac9cd5db4cae,26525,6514b8ad8028c9f2cc2374ded245783f,2017-05-16 13:10:30,110adb18eeba9a0984478ee3c9302e16,179.49,9.45
8,f54a9f0e6b351c431402b8461ea51999,39382392765b6dc74812866ee5ee92a7,99655,76c6e866289321a7c93b82b54852dc33,2017-01-23 18:29:09,110adb18eeba9a0984478ee3c9302e16,179.49,9.45
9,31ad1d1b63eb9962463f764d4e6e0c9d,299905e3934e9e181bfb2e164dd4b4f8,18075,e69bfb5eb88e0ed6a785585b27e16dbf,2017-07-29 11:55:02,110adb18eeba9a0984478ee3c9302e16,179.49,9.45


## 1. 查詢熱銷商品品類

In [ ]:
df_top_products_by_date = duckdb.query(f"""
SELECT
    CAST(order_purchase_timestamp AS DATE) AS order_date,
    product_category_name,
    COUNT(order_items.product_id) AS product_count,
    SUM(order_items.price + order_items.freight_value) AS total_revenue
FROM
    '{order_items}' AS order_items
JOIN
    '{products}' AS products
ON
    order_items.product_id = products.product_id
JOIN
    '{orders}' AS orders
ON
    order_items.order_id = orders.order_id
GROUP BY
    order_date, product_category_name
ORDER BY
    total_revenue DESC, order_date

""").to_df()

df_top_products_by_date


,order_date,product_category_name,product_count,total_revenue
0,2017-11-24,cama_mesa_banho,195,21317.39
1,2017-11-24,relogios_presentes,75,18992.86
2,2017-08-29,pcs,15,17561.05
3,2017-11-24,moveis_decoracao,136,14319.95
4,2017-09-29,telefonia_fixa,8,13664.08
...,...,...,...,...
18985,2017-05-17,fashion_underwear_e_moda_praia,1,13.68
18986,2017-06-12,None,1,12.89
18987,2018-05-23,consoles_games,1,12.89
18988,2017-05-01,artes,1,12.22


In [ ]:
# 下載成 csv file
from google.colab import files
csv_file = 'top_products_by_date.csv'
df_top_products_by_date.to_csv(csv_file, index=False)

# 下載檔案
files.download(csv_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 2. 查詢地區熱銷產品

In [ ]:
# 把重複的geolocation_zip_code_prefix去掉，避免在join的時候duplicate
df_top_products_by_state = duckdb.query(f"""
WITH dedup_geolocation AS (
    SELECT DISTINCT
        geolocation_zip_code_prefix,
        MAX(geolocation_state) AS geolocation_state
    FROM
        '{geolocation}'
    GROUP BY
        geolocation_zip_code_prefix
)

SELECT
    CAST(order_purchase_timestamp AS DATE) AS order_date,
    CASE WHEN
      geolocation.geolocation_state IS NOT NULL THEN geolocation.geolocation_state
      ELSE 'Unknown'
    END AS geolocation_state,
    products.product_category_name,
    COUNT(order_items.product_id) AS product_count,
    SUM(order_items.price + order_items.freight_value) AS total_revenue


FROM
    '{orders}' AS orders
JOIN
    '{customers}' AS customers
ON
    orders.customer_id = customers.customer_id

LEFT JOIN
    dedup_geolocation AS geolocation
ON
    customers.customer_zip_code_prefix = geolocation.geolocation_zip_code_prefix

JOIN
    '{order_items}' AS order_items
ON
    orders.order_id = order_items.order_id

JOIN
    '{products}' AS products
ON
    order_items.product_id = products.product_id
GROUP BY
    order_date, geolocation.geolocation_state, products.product_category_name
ORDER BY
    total_revenue DESC;

""").to_df()

df_top_products_by_state


,order_date,geolocation_state,product_category_name,product_count,total_revenue
0,2017-09-29,RJ,telefonia_fixa,8,13664.08
1,2017-11-24,SP,cama_mesa_banho,79,8150.30
2,2017-04-01,RJ,eletroportateis,2,7571.63
3,2018-07-15,ES,telefonia_fixa,4,7274.88
4,2017-11-24,RJ,relogios_presentes,16,6999.87
...,...,...,...,...,...
56917,2017-05-06,PR,fashion_bolsas_e_acessorios,1,13.44
56918,2018-05-23,MG,consoles_games,1,12.89
56919,2017-06-12,PR,None,1,12.89
56920,2017-05-01,SP,artes,1,12.22


In [ ]:
# 下載成 csv file
from google.colab import files
csv_file = 'top_products_by_state.csv'
df_top_products_by_state.to_csv(csv_file, index=False)

# 下載檔案
files.download(csv_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>